# Test utils

## import module

In [1]:
%run ../config_jupyter.ipynb
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

{'commit_hash': '523ed2fe5',
 'commit_source': 'installation',
 'default_encoding': 'cp1252',
 'ipython_path': 'C:\\Users\\Jesus\\Anaconda3\\lib\\site-packages\\IPython',
 'ipython_version': '7.2.0',
 'os_name': 'nt',
 'platform': 'Windows-10-10.0.17134-SP0',
 'sys_executable': 'C:\\Users\\Jesus\\Anaconda3\\python.exe',
 'sys_platform': 'win32',
 'sys_version': '3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit '
                '(AMD64)]'}
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [17]:
from numpy import matrix, array, cos, sin, pi
from py_pol import degrees, np
from py_pol.utils import (rotation_matrix_Jones, params_to_list, charac_angles_2_azimuth_elipt, 
azimuth_elipt_2_charac_angles, comparison, eps)

from py_pol.jones_vector import Jones_vector
from py_pol.jones_matrix import Jones_matrix
from py_pol.stokes import Stokes


## parms_to_list

### Jones 

In [9]:
J1 = Jones_vector('J1')
J1.from_elements(-1+2j,1-2j)
print(J1)

J1 = [(-1+2j); (1-2j)]'


In [7]:
J1.simplify(kind=3,verbose=True);

J1_cond: [1.000+0.000j; -1.000+0.000j]', amplitude: 2.236, angle: 2.03 deg



In [13]:
print(J1.parameters)

parameters of J1:
    intensity        : 10.000 arb.u
    alpha            : 45.000 deg
    delay            : 180.000 deg
    azimuth          : 45.000 deg
    ellipticity angle: 0.000 deg
    a, b             : 0.000  3.162



In [11]:
params_to_list(J1, verbose=True)

(10.000000000000002, 0.7853981633974483, 3.141592653589793, 0.7853981633974483, 6.123233995736766e-17, 0.0, 3.1622776601683795)


(10.000000000000002,
 0.7853981633974483,
 3.141592653589793,
 0.7853981633974483,
 6.123233995736766e-17,
 0.0,
 3.1622776601683795)

### Jones_matrix

In [30]:
M1 = Jones_matrix('M1')


In [31]:
M1.diattenuator_linear(angle=45*degrees, p1=.9, p2=.1)

matrix([[0.5+0.j, 0.4+0.j],
        [0.4+0.j, 0.5+0.j]])

In [32]:
M1.parameters

parameters of M1 @45.00 deg:
    is_homogeneous: True
    delay:          0.000ª
    diattenuation:  0.976

In [33]:
params_to_list(M1, verbose=True)

(0.0, 0.975609756097561, True)


(0.0, 0.975609756097561, True)

### Stokes

In [38]:
S1 = Stokes('S1')
S1.general_charac_angles(
    alpha=22.5*degrees, delay=45*degrees, intensity=1, pol_degree=.5, is_depolarization=False)

matrix([[1.        ],
        [0.35355339],
        [0.25      ],
        [0.25      ]])

In [39]:
S1.parameters

parameters of S1:
    intensity             : 1.000 arb. u.
    amplitudes            : E0x 0.653, E0y 0.271, E0_unpol 0.707 
    degree polarization   : 0.500
    degree linear pol.    : 0.433
    degree   circular pol.: 0.250
    alpha                 : 22.500 deg
    delay                 : 45.000 deg
    azimuth               : 17.632 deg
    ellipticity  angle    : 15.000 deg
    ellipticity  param    : 0.268
    eccentricity          : 0.963
    polarized vector      : [+0.500; +0.354; +0.250; +0.250]'
    unpolarized vector    : [+0.500; +0.000; +0.000; +0.000]'

In [44]:
params_to_list(S1, verbose=True);

(1.0, (0.6532824824381883, 0.2705990500730985, 0.7071067811865476), 0.5, 0.43301270189221935, 0.24999999999999994, 0.39269908169872414, 0.7853981633974482, 0.30773985433519363, 0.26179938779914935, 0.26794919243112264, [0.5, 0.3535533905932738, 0.25, 0.24999999999999994], [0.5, 0.0, 0.0, 0.0])


## Fast method

In [53]:
lista=[(k,v) for k,v in S1.parameters.dict_params.items()]

In [59]:
for l in lista:
    print("{}: {}".format(l[0],l[1]))

intensity: 1.0
amplitudes: (0.6532824824381883, 0.2705990500730985, 0.7071067811865476)
degree_pol: 0.5
degree_linear_pol: 0.43301270189221935
degree_circular_pol: 0.24999999999999994
alpha: 0.39269908169872414
delay: 0.7853981633974482
ellipticity_param: 0.26794919243112264
ellipticity_angle: 0.26179938779914935
azimuth: 0.30773985433519363
eccentricity: 0.9634330440022852
polarized: [0.5, 0.3535533905932738, 0.25, 0.24999999999999994]
unpolarized: [0.5, 0.0, 0.0, 0.0]


## Change of coordinates

In [18]:
N = 20
errors = np.zeros(N)
for ind in range(N):
    (alpha, delta) = np.random.rand(2)
    alpha = alpha*90*degrees
    delta = delta*360*degrees
    solution = np.array([alpha, delta])
    (fi, chi) = charac_angles_2_azimuth_elipt(alpha, delta)
    proposal = azimuth_elipt_2_charac_angles(fi, chi)
    proposal = np.array(proposal)
    errors[ind] = comparison(proposal, solution, eps)
print(errors)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [23]:
N = 20
errors = np.zeros(N)
for ind in range(N):
    (azimuth, ellipticity) = np.random.rand(2)
    azimuth = azimuth*180*degrees
    ellipticity = (1-2*ellipticity)*45*degrees
    solution = np.array([azimuth, ellipticity])
    (alpha, delay) = azimuth_elipt_2_charac_angles(azimuth, ellipticity) 
    proposal = charac_angles_2_azimuth_elipt(alpha, delay)
    proposal = np.array(proposal)
    errors[ind] = comparison(proposal, solution, eps)
print(errors)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
